In [1]:
import numpy as np
import pandas as pd

## Constants

In [2]:
VOCABULARY_SIZE = 2500

TRAINING_DATA = 'SpamData/02_Training/train-data.txt'
TESTING_DATA = 'SpamData/02_Training/test-data.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'


In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA, delimiter=' ',dtype='int')
sparse_test_data = np.loadtxt(TESTING_DATA, delimiter=' ',dtype='int')

<class 'FileNotFoundError'>: SpamData/02_Training/train-data.txt not found.

In [4]:
sparse_train_data.shape

<class 'NameError'>: name 'sparse_train_data' is not defined

In [5]:
sparse_test_data.shape

<class 'NameError'>: name 'sparse_test_data' is not defined

In [6]:
sparse_train_data[:5] # columns =  doc_id, word_id, category, occurence

<class 'NameError'>: name 'sparse_train_data' is not defined

In [7]:
print(f'Number of rows in training file : {sparse_train_data.shape[0]}')
print(f'Number of rows in testing file : {sparse_test_data.shape[0]}')

<class 'NameError'>: name 'sparse_train_data' is not defined

In [8]:
print(f' Number of emails in training file : {np.unique(sparse_train_data[: , 0]).size}')
print(f' Number of emails in testing file : {np.unique(sparse_test_data[: , 0]).size}')

<class 'NameError'>: name 'sparse_train_data' is not defined

## Creating empty dataframe

In [9]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0 , VOCABULARY_SIZE))

In [10]:
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:,0])
index_names

<class 'NameError'>: name 'sparse_train_data' is not defined

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.head()

<class 'NameError'>: name 'index_names' is not defined

In [14]:
full_train_data.fillna(value=0,inplace=True)

<class 'NameError'>: name 'full_train_data' is not defined

## Creating Full Matrix from sparse matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    '''
    returns full matrix which contains occurences of each word in vocab in single row 
    sparse_matrix -- input matrix (np array)
    nr_words -- size of vocabulary(2500)
    doc_idx -- DOC_ID index in sparse matrix(0)
    word_idx -- 'WORD_ID' index in sparse_matrix(1)
    cat_idx -- 'CATEGORY' index (2)
    freq_idx -- 'OCCURENCES' index (3)
    '''
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0 , VOCABULARY_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):  # iterating through all rows in sparse matrix and extracting word occurences
        doc_nr = sparse_matrix[i][doc_idx] # doc_idx = 0 first column
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurence
        
    full_matrix.set_index('DOC_ID',inplace=True)  
    return full_matrix

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data,VOCABULARY_SIZE)

<class 'NameError'>: name 'sparse_train_data' is not defined

In [17]:
full_train_data.head()  # most frequent words are at begining


#DOC_ID : emails id of each email in dataset
#CATEGORY : wether email is spam or not   1 : spam    0 : legit
#columns 0 1 2 3 4 .......2499 indicates word_id of top 2500 words in vocabulary
#each box inside table indicates number of occurance of word in particular email

<class 'NameError'>: name 'full_train_data' is not defined

## Training Naive Bayes Classifier

## calculating probability of spam 

In [18]:
spam_prob = full_train_data['CATEGORY'].sum() / full_train_data['CATEGORY'].size
print(' Probability of email being spam ' ,round(spam_prob,3))

<class 'NameError'>: name 'full_train_data' is not defined

In [19]:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATEGORY']
full_train_features.head()  #excluding category

<class 'NameError'>: name 'full_train_data' is not defined

In [20]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

<class 'NameError'>: name 'full_train_features' is not defined

In [21]:
total_wordcounts = email_lengths.sum()
total_wordcounts

<class 'NameError'>: name 'email_lengths' is not defined

In [22]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

<class 'NameError'>: name 'email_lengths' is not defined

In [23]:
spam_wordcount = spam_lengths.sum()
spam_wordcount

<class 'NameError'>: name 'spam_lengths' is not defined

In [24]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

<class 'NameError'>: name 'email_lengths' is not defined

In [25]:
ham_wordcount = ham_lengths.sum()
ham_wordcount

<class 'NameError'>: name 'ham_lengths' is not defined

In [26]:
print(f'Avg number of words in spam emails : {spam_wordcount / spam_lengths.shape[0]}')
print(f'Avg number of words in ham emails : {ham_wordcount / ham_lengths.shape[0]}')

<class 'NameError'>: name 'spam_wordcount' is not defined

## Summing the Tokens Occuring in spam

In [27]:
full_train_features.shape

<class 'NameError'>: name 'full_train_features' is not defined

In [28]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

<class 'NameError'>: name 'full_train_features' is not defined

In [29]:
# Calculating number of time each word appeared in all spam emails

summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # +1 to avoid zero by error while calculating probability
summed_spam_tokens.shape    # axis = 0 to sum columns

<class 'NameError'>: name 'train_spam_tokens' is not defined

In [30]:
summed_spam_tokens.head()

<class 'NameError'>: name 'summed_spam_tokens' is not defined

## Summing tokens occuring in ham emails

In [31]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

<class 'NameError'>: name 'full_train_features' is not defined

In [32]:
# Calculating number of time each word appeared in all ham emails

summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 # +1 to avoid zero by error while calculating probability
summed_ham_tokens.shape    # axis = 0 to sum columns

<class 'NameError'>: name 'train_ham_tokens' is not defined

## p( token | spam ) : Probabilty that token ocuurs given email is spam

In [33]:
prob_tokens_spam = summed_spam_tokens / (spam_wordcount + VOCABULARY_SIZE)  #devide by VOCABULARY_SIZE coz we added 1 to 
                                                                                    #summed_spam tokens

<class 'NameError'>: name 'summed_spam_tokens' is not defined

In [34]:
prob_tokens_spam[:5]

<class 'NameError'>: name 'prob_tokens_spam' is not defined

In [35]:
prob_tokens_spam.sum()

<class 'NameError'>: name 'prob_tokens_spam' is not defined

## p( token | nonspam ) : Probabilty that token ocuurs given email is non-spam

In [36]:
prob_tokens_ham = summed_ham_tokens / (ham_wordcount + VOCABULARY_SIZE)
prob_tokens_ham.sum()

<class 'NameError'>: name 'summed_ham_tokens' is not defined

## p(token) : probability that token occurs

In [37]:
prob_token_all = full_train_features.sum(axis=0) / total_wordcounts
prob_token_all

<class 'NameError'>: name 'full_train_features' is not defined

## Saving trained model

In [38]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_token_all)

<class 'NameError'>: name 'prob_tokens_spam' is not defined

## Prepare test data

In [39]:
sparse_test_data.shape

<class 'NameError'>: name 'sparse_test_data' is not defined

In [40]:
%%time
full_test_data = make_full_matrix(sparse_test_data,nr_words=VOCABULARY_SIZE)

<class 'NameError'>: name 'sparse_test_data' is not defined

In [41]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

<class 'NameError'>: name 'full_test_data' is not defined

In [42]:
X_test.shape

<class 'NameError'>: name 'X_test' is not defined

In [43]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,X_test)

<class 'NameError'>: name 'y_test' is not defined